<a href="https://colab.research.google.com/github/ArshiAbolghasemi/AI-UT/blob/main/neural_network/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this project, we will utilize a text dataset to demonstrate advanced natural language processing techniques. Our approach will involve initial preprocessing of the text, followed by the application of Word2Vec and neural networks to effectively analyze and interpret the data. Through these methods, we aim to uncover meaningful patterns and insights within the text.

# Imports and Settings

## Imports

In [ ]:
import os

from google.colab import drive

## Settings

In [ ]:
DRIVE_MOUNTED_PATH=os.path.join(os.getcwd(), 'drive/')
TWITTER_SUICIDAL_DATASET_PATH=os.path.join(DRIVE_MOUNTED_PATH, 'MyDrive/AI-UT/neural_network/')

## Mounting to Google Drive

In [ ]:
drive.mount(DRIVE_MOUNTED_PATH)

Mounted at /content/drive/
